# 📘 [Upstage] AI Agent QA 시나리오 데이터셋 구축 

## 콘텐츠 라이선스

저작권: <b><font color='blue'>© 2025 Upstage Co., Ltd. All rights reserved.</font></b>

<b><font color='red'>⚠️ WARNING:</font></b> 본 교육용 코드 및 자료의 모든 지식재산권은 업스테이지에 귀속됩니다.  
본 콘텐츠를 사전 서면 동의 없이 **외부로 유출, 복제, 수정, 배포, 게시하는 행위는 엄격히 금지**됩니다.


## 🎯 프로젝트 개요
본 프로젝트는 **한국 Public API**를 활용하여 도구를 적절히 활용하는 AI Agent 시나리오 데이터셋을 구축합니다.
모델의 Agentic 능력 향상을 위한 고품질 학습 데이터셋을 구축하여, 모델 학습을 위한 데이터 구축 방법론에 대해 학습합니다.  


### 핵심 목표
- **Multi-Turn Tool 호출 시나리오** 구축합니다.
- **오류 복구, 조건부 선택** 등 Agentic Reasoning을 요구하는 내용을 반영합니다. 
- **BFCL 기준 검증** 통과한 고품질 시나리오 데이터셋을 구축하여, 현업에서 실제 평가 벤치마크를 구축하는 방법을 학습합니다. 

---
## 📋 프로젝트 진행 단계

### 1/ 도메인 선정
- 10개 도메인 중 1개 선택 (정부 & 공공기관, 게임 & 엔터테인먼트, 소셜 & 커뮤니케이션, 지도/위치, 네이버, 카카오, 쇼핑, 음식/의료, 날씨/환경, 미디어 & 콘텐츠)

### 2/ Tool 정의
- 최소 2개 이상의 독립적인 Tool 설계
- Real-world 시나리오 활용 가능
- Public API 기반

### 3/ QA 시나리오 데이터 구축
- **조합 1**: Single-turn & Multi-tool-use (최소 2개)
- **조합 2**: Multi-turn & Multi-tool-use (최소 2개)

### 4/ Solar Pro 2 API 호출
- 실제 Solar Pro 2 API로 합성 데이터 생성
- 재시도 로직 포함

### 5/ 정량 평가
- Correct Function Name
- Valid Arguments  
- No Hallucinated Calls

### 6/ 정성 평가 
아래 기준에 따라 정성 평가를 진행합니다.
- 시나리오 적합성 
- 창의성 및 다양성 
- 구체성
- 실행 가능성 
- 윤리·안전성


### 7/ 최종 데이터셋
- 평가 통과한 시나리오만 포함
- JSONL, JSON, CSV 형식으로 저장

---

## 📊 데이터셋 구성

| 조합 | Turn | Tool 사용 | 최소 개수 |
|------|------|-----------|----------|
| 조합 1 | Single-turn | Multi-tool-use | 2개 |
| 조합 2 | Multi-turn | Multi-tool-use | 2개 |

**총 최소**: 4개 시나리오

## 실험 평가 기준

평가 설계 기준 
- 아래 기준 정도로 실험하는 것을 권장 드립니다.

1. 조합 1 ㅣSingle Turn &  Multi Tool Use
   * 정량 50% 
   * 정성 50% 

2. 조합 2 | Multi Turn &  Multi Tool Use
   * 정량 : 30% 
   * 정성 : 70%
     * Multi Turn의 경우, 오류 복구 시나리오를 포함한 데이터의 정성평가의 비중을 더 높여 프로젝트를 진행해보세요! 


---

## 📚 참고 자료
- [활용 데이터 : Public API](https://github.com/yybmion/public-apis-4Kr)
- [BFCL 리더보드](https://github.com/ShishirPatil/gorilla)
- [Nemotron 시나리오 데이터](https://huggingface.co/datasets/nvidia/Nemotron-Post-Training-Dataset-v1)
- [Upstage Console](https://developers.upstage.ai/)

---


# 0️⃣ 환경 설정

이 섹션에서는 프로젝트 실행에 필요한 라이브러리와 BFCL을 설치합니다.

### 필요한 패키지
- `openai`: Solar Pro 2 API 호출
- `requests`: HTTP 요청
- `python-dotenv`: 환경변수 관리
- `tqdm`, `pandas`, `numpy`: 데이터 처리
- BFCL (Berkeley Function Call Leaderboard): 평가 프레임워크

In [1]:
# ============================================================
# 필수 라이브러리 임포트
# ============================================================
import os, sys, subprocess, json
import uuid
import time
import random
import csv
import requests
import urllib.request
import pandas as pd


import re
from openai import OpenAI  # Solar Pro 2 API 호출용 (openai==1.52.2)
from pathlib import Path
from datetime import datetime
from dotenv import load_dotenv
from typing import List, Dict, Any, Optional


print("✅ 라이브러리 임포트 완료")

# ============================================================
# requirements.txt 생성
# ============================================================
# req = """
# python-dotenv>=1.0.1
# requests>=2.32.3
# tqdm>=4.66.5
# rich>=13.8.1
# pandas>=2.2.2
# numpy>=1.26.4
# datasets>=2.21.0
# huggingface-hub>=0.24.6
# """
# with open("requirements.txt", "w", encoding="utf-8") as f:
#     f.write(req.strip() + "\n")

# print("✅ requirements.txt 생성 완료")

✅ 라이브러리 임포트 완료


In [2]:
# ============================================================
# Solar Pro 2 API Key 설정
# ============================================================
# 
# 📌 API Key 발급 방법:
# 1. https://console.upstage.ai/ 접속
# 2. 회원가입 / 로그인
# 3. Dashboard > API Keys 메뉴에서 새 키 발급
# 4. 아래에 발급받은 키를 입력하세요
#
# ⚠️ 보안 주의사항:
# - API Key를 코드에 직접 입력하는 것은 보안상 위험합니다
# - .env 파일 또는 환경변수 사용을 권장합니다
# - GitHub 등에 업로드 시 키가 노출되지 않도록 주의하세요
# ============================================================

# 방법 1: 직접 입력 (테스트용)
UPSTAGE_API_KEY = "up_UqkrX33H3GOk2EeAJroP0sXzFDhs6"  # ← 여기에 발급받은 키를 입력하세요

# 방법 2: .env 파일 사용 (권장)
# .env 파일을 만들고 다음 내용 추가:
# UPSTAGE_API_KEY=your_actual_api_key_here
env_file = Path(".env")
if env_file.exists():
    load_dotenv()
    UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY", UPSTAGE_API_KEY)
    print("✅ .env 파일에서 API 키를 로드했습니다")
else:
    if UPSTAGE_API_KEY == "YOUR_API_KEY_HERE":
        print("⚠️  API Key를 설정해주세요!")
        print("   위 코드에서 UPSTAGE_API_KEY 값을 실제 키로 변경하세요")
    else:
        print("✅ API Key 설정 완료")

# API 기본 설정
BASE_URL = "https://api.upstage.ai/v1"
print(f"✅ Solar Pro 2 API 엔드포인트: {BASE_URL}")


✅ .env 파일에서 API 키를 로드했습니다
✅ Solar Pro 2 API 엔드포인트: https://api.upstage.ai/v1


In [3]:
# ============================================================
# 출력 디렉토리 생성
# ============================================================
# outputs: 최종 데이터셋 파일 저장 (JSONL, JSON)
# artifacts: 평가 리포트 저장 (CSV)
os.makedirs("outputs", exist_ok=True)
os.makedirs("artifacts", exist_ok=True)

print("✅ 출력 디렉토리 생성 완료")
print("   - outputs/ : 최종 데이터셋 파일")
print("   - artifacts/ : BFCL 평가 리포트")

✅ 출력 디렉토리 생성 완료
   - outputs/ : 최종 데이터셋 파일
   - artifacts/ : BFCL 평가 리포트


# api 호출

## 카카오

In [14]:
ACCESS_TOKEN = "${dcfce2ab5c911dae25ee1e1c6273e335}"

# 모든 요청에 공통으로 사용될 헤더
headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}"
    # Content-Type은 requests 라이브러리가 data 파라미터를 사용할 때
    # 'application/x-www-form-urlencoded'로 자동 설정해줍니다.
    # 문서를 따라 명시적으로 추가해도 무방합니다.
    # "Content-Type": "application/x-www-form-urlencoded;charset=utf-8"
}

In [15]:
def send_memo_default(access_token):
    """나에게 기본 템플릿(텍스트) 메시지 보내기"""
    url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"
    
    headers = {"Authorization": f"Bearer {access_token}"}

    # 텍스트 템플릿
    template = {
        "object_type": "text",
        "text": "Python에서 보내는 '나에게' 테스트 메시지입니다.",
        "link": {
            "web_url": "https://developers.kakao.com",
            "mobile_web_url": "https://developers.kakao.com"
        },
        "button_title": "자세히 보기"
    }
    
    # 템플릿 객체를 JSON 문자열로 변환
    data = {
        "template_object": json.dumps(template)
    }

    try:
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status() # 오류 발생 시 예외 발생
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        if e.response:
            print(f"Response: {e.response.json()}")
        return None

# 함수 호출
result = send_memo_default(ACCESS_TOKEN)
if result:
    print("나에게 보내기 성공:", result)

Error: 401 Client Error: Unauthorized for url: https://kapi.kakao.com/v2/api/talk/memo/default/send


## 네이버

In [4]:
import os
import sys
import urllib.request
client_id = "SpyLU6veoNcp6flgp0kN"
client_secret = "afatAxdR2t"

url = "https://openapi.naver.com/v1/datalab/shopping/category/keywords"

# 1. body를 수동 문자열이 아닌 Python 딕셔너리로 선언
# 네이버 데이터랩 API (쇼핑/카테고리별 키워드) 요청 바디 (dict)
body_dict = {
    # (필수) 조회 기간 시작 날짜 (yyyy-mm-dd 형식)
    "startDate": "2025-10-01",
    
    # (필수) 조회 기간 종료 날짜 (yyyy-mm-dd 형식)
    "endDate": "2025-10-31",
    
    # (필수) 구간 단위 (date: 일간, week: 주간, month: 월간)
    "timeUnit": "month",
    
    # (필수) 쇼핑 분야 코드 (네이버쇼핑 URL의 cat_id 값)
    # "category": "50000000",
    
# (참고: '패션잡화'(50000002) 카테고리 사용. '신발'의 상위 카테리입니다.)
    "category": "10000110", 
    
    # (필수) 검색 키워드 그룹 (요청하신 4개 모델)
    "keyword": [
        {
            "name": "뉴발란스 2002",
            "param": [ "뉴발란스 2002" ] 
        },
        {
            "name": "뉴발란스 992",
            "param": [ "뉴발란스 992" ]
        },
        {
            "name": "뉴발란스 574",
            "param": [ "뉴발란스 574" ]
        },
        {
            "name": "뉴발란스 530",
            "param": [ "뉴발란스 530" ]
        }
    ],
    
    # (선택) 전체 기기, 성별, 연령
    "device": "",
    "gender": "",
    "ages": []
}

# 4. 딕셔너리를 JSON 문자열로 변환
body = json.dumps(body_dict)

# --- (기존 코드와 동일) ---
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")

# body.encode("utf-8")는 동일하게 사용
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()

if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{"startDate":"2025-10-01","endDate":"2025-10-31","timeUnit":"month","results":[{"title":"뉴발란스 2002","keyword":["뉴발란스2002"],"data":[]},{"title":"뉴발란스 992","keyword":["뉴발란스992"],"data":[]},{"title":"뉴발란스 574","keyword":["뉴발란스574"],"data":[]},{"title":"뉴발란스 530","keyword":["뉴발란스530"],"data":[]}]}


In [ ]:
# 네이버 쇼핑 검색

# 1. 검색 조건 설정
query = "뉴발란스 530"       # 검색할 단어
display = 10               # 검색 결과 출력 건수
start = 1                  # 검색 시작 위치
sort = "dsc"               # 정렬 옵션: sim(유사도순), date(날짜순), asc(가격 오름차순), dsc(가격 내림차순)

# 2. 검색어 UTF-8 인코딩
encText = urllib.parse.quote(query)

# 3. URL 생성 (shop.json으로 변경 및 파라미터 추가)
url = f"https://openapi.naver.com/v1/search/shop.json?query={encText}&display={display}&start={start}&sort={sort}"

# 4. Request 객체 생성 및 헤더 추가
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id", client_id)
request.add_header("X-Naver-Client-Secret", client_secret)

# 5. API 요청
try:
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    
    if rescode == 200:
        response_body = response.read()
        
        # 6. JSON 응답 파싱 및 보기 좋게 출력
        result = json.loads(response_body.decode('utf-8'))
        print(json.dumps(result, indent=4, ensure_ascii=False))
        
        # 7. (참고) 상품명과 최저가만 리스트로 출력
        print("\n--- 간략한 상품 리스트 ---")
        for item in result['items']:
            # <b> 태그 제거 (간단한 replace 사용)
            title = item['title'].replace('<b>', '').replace('</b>', '')
            print(f"상품명: {title} | 최저가: {item['lprice']}원")
            
    else:
        print(f"Error Code: {rescode}")

except urllib.error.HTTPError as e:
    # 400, 401, 403, 404, 500 등 에러 코드 처리
    print(f"HTTP Error Code: {e.code}")
    print(f"Error Message: {e.read().decode('utf-8')}")
except Exception as e:
    print(f"An error occurred: {e}")

{
    "lastBuildDate": "Wed, 12 Nov 2025 15:44:49 +0900",
    "total": 47343,
    "start": 1,
    "display": 10,
    "items": [
        {
            "title": "<b>뉴발란스 530</b> 레인 클라우드 섬머 포그9",
            "link": "https://smartstore.naver.com/main/products/11916068443",
            "image": "https://shopping-phinf.pstatic.net/main_8946057/89460579052.jpg",
            "lprice": "10428000",
            "hprice": "",
            "mallName": "HIGHTHROW",
            "productId": "89460579052",
            "productType": "2",
            "brand": "",
            "maker": "",
            "category1": "패션잡화",
            "category2": "남성신발",
            "category3": "스니커즈",
            "category4": ""
        },
        {
            "title": "<b>뉴발란스 530</b> 스틸 그레이 MR530KA.",
            "link": "https://www.11st.co.kr/connect/Gateway.tmall?method=Xsite&prdNo=6295454820&tid=1000000061",
            "image": "https://shopping-phinf.pstatic.net/main_5746720/57467204423.jpg",
            "lpri

## naver 검색

In [19]:
client_id = "Jy7UambNoEYDgkfyuEel"
client_secret = "ig9mbNcHgI"

In [23]:
encText = urllib.parse.quote("입법")
url_news = "https://openapi.naver.com/v1/search/news.xml?query=" + encText # JSON 결과
# url_blog = "https://openapi.naver.com/v1/search/blog?query=" + encText # JSON 결과

# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
request = urllib.request.Request(url_news)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

<?xml version="1.0" encoding="UTF-8"?><rss version="2.0"><channel><title>Naver Open API - news ::&apos;입법&apos;</title><link>https://search.naver.com</link><description>Naver Search Result</description><lastBuildDate>Mon, 10 Nov 2025 17:25:07 +0900</lastBuildDate><total>2064720</total><start>1</start><display>10</display><item><title>윤 전 대통령 &amp;quot;자유로운 분위기에서 일단 출동&amp;quot;…방첩사 간부 증언 반박</title><originallink>http://mbn.mk.co.kr/pages/news/newsView.php?category=mbn00006&amp;news_seq_no=5153429</originallink><link>https://n.news.naver.com/mnews/article/057/0001918026?sid=100</link><description>이에 윤 전 대통령은 &amp;quot;계엄이란 건 유사 군정과 비슷해서 계엄이 선포되면 계엄 당국이 &lt;b&gt;입법&lt;/b&gt;부를 제외하고는 행정·사법 업무를 직접 관장하거나 지휘·감독할 권한이 법에 의해 주어진다&amp;quot;며 &amp;quot;정부 부처에 들어가서 수사... </description><pubDate>Mon, 10 Nov 2025 17:24:00 +0900</pubDate></item><item><title>방첩사 장교, 윤 재판에서 &amp;quot;방첩사 내부에도 계엄 저항 세력 있었다는 ...</title><originallink>https://news.sbs.co.kr/news/endPage.do?news_id=N1008325875&amp;plink=ORI&a

# 정부

## 교통

In [4]:
service_key = '5a7d676a324d70d7f75eef653374fbca16be8029b54c75754f081996e996f374'

In [5]:
BASE_URL = "http://apis.data.go.kr/B551457/run/v2"
ENDPOINT = "/travelerTrainRunPlan2"  # 이전에 제공해주신 엔드포인트

url = BASE_URL + ENDPOINT

params = {
    # --- [필수] 기본 파라미터 ---
    'serviceKey': service_key,  # 공공데이터포털 인증키
    
    # --- [선택] 페이징 파라미터 ---
    'pageNo': '1',              # 페이지 번호 (기본값: 1)
    'numOfRows': '10',          # 한 페이지 당 결과 수 (기본값: 10)
    
    # --- [선택] 응답 형식 ---
    'returnType': 'json',       # 응답 데이터 타입 (JSON 또는 XML)
    
    # --- [선택] 검색 조건 (cond) ---
    
    # 운행일자 (YYYYMMDD 형식)
    'cond[run_ymd::GTE]': '20251111',  # [이상] 운행일자가 이 날짜(포함) 이후인 데이터
    'cond[run_ymd::LTE]': '20251130',  # [이하] 운행일자가 이 날짜(포함) 이전인 데이터
    
    # 출발역
    'cond[dptre_stn_cd::EQ]': 'NAT010000', # [일치] 출발역 코드 (예: 서울역 코드)
    'cond[dptre_stn_nm::EQ]': '서울',      # [일치] 출발역 명칭
    
    # 도착역
    'cond[arvl_stn_cd::EQ]': 'NAT011668',  # [일치] 도착역 코드 (예: 부산역 코드)
    'cond[arvl_stn_nm::EQ]': '부산',      # [일치] 도착역 명칭
}
response = requests.get(url, params=params)

In [6]:
response.content

b'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body>\r\n<center><h1>403 Forbidden</h1></center>\r\n<hr><center>nginx/1.18.0 (Ubuntu)</center>\r\n</body>\r\n</html>\r\n'

## 부산 여행정보

In [7]:
url = 'http://apis.data.go.kr/6260000/FoodService/getFoodKr'
params ={'serviceKey' : service_key, 'pageNo' : '1', 'numOfRows' : '10', 'resultType' : 'json'
        #  , 'UC_SEQ' : '' 
         }

response = requests.get(url, params=params)
response.json()

{'getFoodKr': {'header': {'code': '00', 'message': 'NORMAL_CODE'},
  'item': [{'UC_SEQ': 70,
    'MAIN_TITLE': '만드리곤드레밥',
    'GUGUN_NM': '강서구',
    'LAT': 35.177387,
    'LNG': 128.95245,
    'PLACE': '만드리곤드레밥',
    'TITLE': '만드리곤드레밥',
    'SUBTITLE': '',
    'ADDR1': '강서구 공항앞길 85번길 13',
    'ADDR2': '',
    'CNTCT_TEL': '051-941-3669',
    'HOMEPAGE_URL': '',
    'USAGE_DAY_WEEK_AND_TIME': '10:00-20:00\n(19:50 라스트오더)',
    'RPRSNTV_MENU': '돌솥곤드레정식, 단호박오리훈제',
    'MAIN_IMG_NORMAL': 'https://www.visitbusan.net/uploadImgs/files/cntnts/20191209162810545_ttiel',
    'MAIN_IMG_THUMB': 'https://www.visitbusan.net/uploadImgs/files/cntnts/20191209162810545_thumbL',
    'ITEMCNTNTS': '곤드레밥에는 일반적으로 건조 곤드레나물이 사용되는데,\n이곳은 생 곤드레나물을 사용하여 돌솥밥을 만든다.\n된장찌개와 함께 열 가지가 넘는 반찬이 제공되는 돌솥곤드레정식이 인기있다\n'},
   {'UC_SEQ': 77,
    'MAIN_TITLE': '민물가든',
    'GUGUN_NM': '강서구',
    'LAT': 35.16055,
    'LNG': 128.89468,
    'PLACE': '민물가든',
    'TITLE': '민물가든',
    'SUBTITLE': '민물가든',
    'ADDR1': '강서구 둔치중앙길5(봉림동) ',

## 교통량

In [38]:
AUTH_KEY = '9406198835'  # 👈 여기에 실제 인증키를 입력하세요.

# 제공된 Request URL
URL = "https://data.ex.co.kr/openapi/odtraffic/trafficAmountByUnit"

# 2. 요청 파라미터 설정 (제공된 '요청변수' 기반)
# ---------------------------------------------------------------
params = {
    # --- [필수] ---
    'key': AUTH_KEY,
    'type': 'json',             # 검색결과 포맷 (json 또는 xml)
    'sumTmUnitTypeCode': '3',     # 집계시간단위 (1:5분, 2:15분, 3:1시간, 4:1일)
    'stdHour': '10',            # 기준 시 (예: 10시)
    
    # --- [선택] ---
    'numOfRows': '1000',          # 한 페이지당 출력 건수
    'pageNo': '10',              # 출력 페이지 번호
}

In [39]:
response = requests.get(URL, params=params)

In [40]:
result_df = pd.DataFrame(response.json())
result_df.head(1)

,count,list,pageNo,numOfRows,pageSize,message,code
0,6282,"{'unitName': '목천', 'pageNo': None, 'numOfRows'...",10,99,64,인증키가 유효합니다.,SUCCESS


In [42]:
new_df = pd.DataFrame(result_df['list'].tolist())
new_df

,unitName,pageNo,numOfRows,unitCode,inoutName,trafficAmout,centerCode,centerName,stdDate,sumTmUnitTypeCode,startEndStdTypeCode,tcsCarTypeCd,stdHour,brofCode,brofName,tcsCarTypeNm,tcsCarTypeGrpNm,tcsCarTypeGrpCd
0,목천,None,None,110,출구,55,903,대전충남본부,20251112,None,None,3,10,410100,천안,3종,대형차,3
1,목천,None,None,110,출구,46,903,대전충남본부,20251112,None,None,4,10,410100,천안,4종,대형차,3
2,목천,None,None,110,입구,39,903,대전충남본부,20251112,None,None,4,10,410100,천안,4종,대형차,3
3,목천,None,None,110,입구,38,903,대전충남본부,20251112,None,None,5,10,410100,천안,5종,대형차,3
4,목천,None,None,110,출구,48,903,대전충남본부,20251112,None,None,5,10,410100,천안,5종,대형차,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,신탄진,None,None,113,입구,44,903,대전충남본부,20251112,None,None,4,10,410200,대전,4종,대형차,3
95,신탄진,None,None,113,입구,87,903,대전충남본부,20251112,None,None,5,10,410200,대전,5종,대형차,3
96,신탄진,None,None,113,출구,72,903,대전충남본부,20251112,None,None,5,10,410200,대전,5종,대형차,3
97,신탄진,None,None,113,입구,31,903,대전충남본부,20251112,None,None,6,10,410200,대전,6종,소형차,1


## 0-1 Solar Pro 2 호출해보기

https://console.upstage.ai/docs/getting-started

In [12]:
# pip install openai
 
client = OpenAI(
    api_key=UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1"
)
 
stream = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "user",
            "content": "Hi, how are you?"
        }
    ],
    stream=True,
)
 
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
 
# Use with stream=False
# print(stream.choices[0].message.content)

Hello! I'm just a large language model, so I don't have feelings, but I'm here and ready to help you. How can I assist you today? Is there something you'd like to talk about or ask? 😊  

*(If you're looking for a joke, fun fact, or specific information—just let me know!)*

# 1️⃣ 도메인 선정 & Tool 정의

이 섹션에서는 프로젝트에 사용할 **도메인을 선택**하고, **최소 2개 이상의 Tool**을 정의합니다.

- **도메인 선택**: 10개 도메인 중 1개 선택
- **Tool 개수**: 최소 2개 이상 정의
- **Tool 정의 시 유의사항**: 
  - 독립적이고 중복 없음
  - Real-world 시나리오에 활용 가능
  - Public API 기반
  - sub-/super-set 관계 없음

#### 📋 선택 가능한 도메인 (10개)

| 번호 | 도메인 | 예시 API |
|------|--------|----------|
| 1 | 정부 & 공공기관 | 공공데이터포털, 국방부 |
| 2 | 게임 & 엔터테인먼트 | 게임 API, 스팀 |
| 3 | 소셜 & 커뮤니케이션 | SNS API, 메신저 |
| 4 | 지도/위치 | 카카오맵, 네이버맵 |
| 5 | 네이버 | 블로그, 뉴스, 지역검색 |
| 6 | 카카오 | 카카오톡, 카카오맵 |
| 7 | 쇼핑 | 쇼핑몰 API, 가격비교 |
| 8 | 음식/의료 | 맛집정보, 병원정보 |
| 9 | 날씨/환경 | 기상청, 에어코리아 |
| 10 | 미디어 & 콘텐츠 | 영화, 음악 API |

### 본 베이스라인의 도메인: 정부 & 공공기관 (방산/국방)

이 노트북에서는 **방산/국방 도메인**을 예시로 사용합니다.

#### 정의된 Tool (2개)

| Tool | 설명 | Public API |
|------|------|------------|
| **defense_company_info** | 국내 방산 업체 정보 조회 | Mock Data (실제: 방위사업청) |
| **rok_weapon_info** | 대한민국 무기체계 개요 조회 | Mock Data (실제: 국방부) |


In [5]:
# ============================================================
# 도메인 선택 (여기를 수정하세요!)
# ============================================================
# 10개 도메인 중 1개를 선택합니다.
# 다른 도메인으로 변경하려면 아래 값을 수정하세요.
# ============================================================

SELECTED_DOMAIN = "정부 & 공공기관 (방산/국방)"

print(f"✅ 선택한 도메인: {SELECTED_DOMAIN}")

✅ 선택한 도메인: 정부 & 공공기관 (방산/국방)


In [6]:
# ============================================================
# Tool 함수 정의
# ============================================================
# 여기서 실제 Tool 함수를 구현합니다.
# 다른 도메인으로 변경하려면 이 셀을 수정하세요.
# ============================================================

# --------------------------
# Tool 1: 방산 업체 정보 조회
# --------------------------
def defense_company_info(query: str,
                         field: Optional[str] = None,
                         limit: int = 10) -> List[Dict[str, Any]]:
    """
    방산 업체(대한민국 중심) 기본 정보 조회
    
    Args:
        query (str): 회사명/키워드 (예: 'Hanwha', '레이더')
        field (str, optional): 특정 필드만 추출 (예: 'careers_url', 'domains')
        limit (int): 결과 최대 개수 (기본값: 10)
    
    Returns:
        List[Dict]: 조회된 업체 정보 리스트
    """
    db = [
        {
            "name": "Hanwha Systems",
            "hq_city": "서울",
            "founded_year": 1977,
            "domains": ["방산전자", "레이더/전자전", "C4ISR", "우주/위성"],
            "subsidiaries": ["한화시스템/ICT", "한화에어로스페이스(관계사)"],
            "careers_url": "https://hcs.hanwha.com/ko/recruit",
            "website": "https://www.hanwhasystems.com"
        },
        {
            "name": "LIG Nex1",
            "hq_city": "성남",
            "founded_year": 1976,
            "domains": ["유도무기", "감시정찰", "전자전", "해양/수중체계"],
            "subsidiaries": [],
            "careers_url": "https://lignex1.recruiter.co.kr",
            "website": "https://www.lignex1.com"
        },
        {
            "name": "KAI",
            "hq_city": "사천",
            "founded_year": 1999,
            "domains": ["항공기", "헬기", "MRO", "우주체계"],
            "subsidiaries": [],
            "careers_url": "https://koreaaero-recruit.applyin.co.kr/jobs/",
            "website": "https://www.koreaaero.com"
        },
        {
            "name": "ADD (국방과학연구소)",
            "hq_city": "대전",
            "founded_year": 1970,
            "domains": ["국방 R&D", "시험평가", "핵심기술개발"],
            "subsidiaries": [],
            "careers_url": "https://www.add.re.kr/board?menuId=MENU00670",
            "website": "https://www.add.re.kr"
        }
    ]

    q = (query or "").lower()
    results = []
    for item in db:
        hay = " ".join([item["name"], item["hq_city"], " ".join(item["domains"])])
        if q in hay.lower():
            if field and field in item:
                results.append({field: item[field], "name": item["name"]})
            else:
                results.append(item)
        if len(results) >= limit:
            break
    return results


# --------------------------
# Tool 2: 대한민국 무기체계 정보 조회
# --------------------------
def rok_weapon_info(keyword: str,
                    category: Optional[str] = None,
                    limit: int = 10) -> List[Dict[str, Any]]:
    """
    대한민국 무기체계 개요(공개/비민감) 정보 조회
    
    Args:
        keyword (str): 무기명/키워드 (예: 'KF-21', '천궁', 'K9')
        category (str, optional): 무기 분류
            - 'aircraft': 항공기
            - 'missile': 미사일
            - 'ship': 함정
            - 'armored': 기갑
            - 'artillery': 포병
            - 'air_defense': 방공
            - 'uav': 무인기
            - 'radar': 레이더
        limit (int): 결과 최대 개수 (기본값: 10)
    
    Returns:
        List[Dict]: 조회된 무기체계 정보 리스트
    """
    db = [
        {
            "name": "KF-21 보라매",
            "category": "aircraft",
            "role": "다목적 전투기(개발 중/시제기 비행)",
            "manufacturer": "KAI(주관) / 국내 다수 협력",
            "intro_year": 2021,
            "notes": "한국형 차세대 전투기 사업(KF-X) 시제 비행 진행"
        },
        {
            "name": "FA-50",
            "category": "aircraft",
            "role": "경공격기/훈련기 파생",
            "manufacturer": "KAI",
            "intro_year": 2013,
            "notes": "수출 다수, 훈련/경공격 임무"
        },
        {
            "name": "K2 흑표",
            "category": "armored",
            "role": "주력전차",
            "manufacturer": "현대로템",
            "intro_year": 2014,
            "notes": "국내 양산/수출형 개발"
        },
        {
            "name": "K9 자주포",
            "category": "artillery",
            "role": "155mm 자주곡사포",
            "manufacturer": "한화에어로스페이스",
            "intro_year": 1999,
            "notes": "국내/해외 다수 운용"
        },
        {
            "name": "천궁(M-SAM)",
            "category": "air_defense",
            "role": "중거리 지대공 유도무기",
            "manufacturer": "LIG Nex1(체계), 한화 등",
            "intro_year": 2015,
            "notes": "한국형 중거리 방공"
        },
        {
            "name": "현무-2",
            "category": "missile",
            "role": "지대지 유도무기(개요 수준)",
            "manufacturer": "국내 개발",
            "intro_year": 2000,
            "notes": "세부 제원은 공개 범위 외"
        },
        {
            "name": "대구급 호위함(FFX Batch-II)",
            "category": "ship",
            "role": "다목적 호위함",
            "manufacturer": "현대중공업/대우조선해양",
            "intro_year": 2018,
            "notes": "대잠/대공/대함 임무"
        },
        {
            "name": "P-ISR 다목적 감시 레이더(예시)",
            "category": "radar",
            "role": "지상/해상 감시 레이더(개요)",
            "manufacturer": "국내 방산 전자",
            "intro_year": 2020,
            "notes": "세부 운용/제원은 비민감 범위만 기술"
        },
        {
            "name": "RQ-101 송골매",
            "category": "uav",
            "role": "전술 정찰 무인기",
            "manufacturer": "국내 개발",
            "intro_year": 2002,
            "notes": "개요 수준 정보"
        }
    ]

    kw = (keyword or "").lower()
    cat_ok = {"aircraft","missile","ship","armored","artillery","air_defense","uav","radar"}
    results = []
    for item in db:
        if category and category not in cat_ok:
            continue
        if category and item["category"] != category:
            continue
        hay = " ".join([item["name"], item["category"], item["role"], item["manufacturer"], item["notes"]])
        if kw in hay.lower():
            results.append(item)
        if len(results) >= limit:
            break
    return results

### Tool 형식

- 아래 형식에 따라 작성합니다.

In [7]:
# {
#   "type": "function",
#   "function": {
#     "name": "<함수명>",
#     "description": "<함수의 목적과 사용법>",
#     "parameters": {
#       "type": "object",
#       "properties": {
#         "<argument명>": {
#           "type": "<자료형>",
#           "description": "<argument 설명>"
#         }
#       },
#       "required": ["<필수 argument 리스트>"]
#     }
#   }
# }

In [8]:
# --------------------------
# 3) Function Calling 메타데이터
# --------------------------

tools = [
    {
        "type": "function",
        "function": {
            "name": "defense_company_info",
            "description": "국내 방산 업체 기본 정보를 조회합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "회사명/키워드 (예: 'Hanwha', '레이더')"},
                    "field": {"type": "string", "description": "특정 필드만 반환(예: 'domains','careers_url')"},
                    "limit": {"type": "integer", "minimum": 1, "maximum": 50, "default": 10}
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
                "name": "rok_weapon_info",
                "description": "대한민국 무기체계 개요(공개/비민감) 정보를 조회합니다.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "keyword": {"type": "string", "description": "무기명/키워드 (예: 'KF-21','천궁')"},
                        "category": {"type": "string", "enum": ["aircraft","missile","ship","armored","artillery","air_defense","uav","radar"]},
                        "limit": {"type": "integer", "minimum": 1, "maximum": 50, "default": 10}
                    },
                    "required": ["keyword"]
                }
        }
    }
]

In [10]:

# ============================================================
# Tool 정의 완료 및 테스트
# ============================================================
print("=" * 60)
print("✅ Tool 정의 완료!")
print("=" * 60)
print(f"선택한 도메인: {SELECTED_DOMAIN}")
print(f"정의된 Tool 개수: {len(tools)}개")
for i, tool in enumerate(tools, 1):
    print(f"  {i}. {tool['function']['name']}")
print("=" * 60)

# Tool 함수 테스트 (예시)
print("\n[테스트 1] defense_company_info(query='레이더')")
ex1 = defense_company_info(query="레이더", field=None, limit=5)
print(f"  결과: {len(ex1)}개 조회됨")
if ex1:
    print(f"  예시: {ex1[0]['name']}")

print("\n[테스트 2] defense_company_info(query='Hanwha', field='careers_url')")
ex2 = defense_company_info(query="Hanwha", field="careers_url", limit=5)
print(f"  결과: {len(ex2)}개 조회됨")
if ex2:
    print(f"  채용 URL: {ex2[0]['careers_url']}")

print("\n[테스트 3] rok_weapon_info(keyword='천궁', category='air_defense')")
ex3 = rok_weapon_info(keyword="천궁", category="air_defense", limit=5)
print(f"  결과: {len(ex3)}개 조회됨")
if ex3:
    print(f"  예시: {ex3[0]['name']} - {ex3[0]['role']}")

print("\n" + "=" * 60)
print("✅ Tool 함수 정상 작동 확인 완료!")
print("=" * 60)

✅ Tool 정의 완료!
선택한 도메인: 정부 & 공공기관 (방산/국방)
정의된 Tool 개수: 2개
  1. defense_company_info
  2. rok_weapon_info

[테스트 1] defense_company_info(query='레이더')
  결과: 1개 조회됨
  예시: Hanwha Systems

[테스트 2] defense_company_info(query='Hanwha', field='careers_url')
  결과: 1개 조회됨
  채용 URL: https://hcs.hanwha.com/ko/recruit

[테스트 3] rok_weapon_info(keyword='천궁', category='air_defense')
  결과: 1개 조회됨
  예시: 천궁(M-SAM) - 중거리 지대공 유도무기

✅ Tool 함수 정상 작동 확인 완료!


# 2️⃣ QA 시나리오 데이터 구축

이 섹션에서는 **Solar Pro 2 API**를 사용하여 시나리오를 자동 생성합니다.

### 📌 생성할 시나리오
1. **Single-turn & Multi-tool-use**: 최소 2개
2. **Multi-turn & Multi-tool-use**: 최소 2개

### 🔄 프로세스
```
사용자 질문 입력
    ↓
Solar Pro 2 API 호출
    ↓
Tool 호출 (자동)
    ↓
Tool 실행 결과 반환
    ↓
최종 답변 생성
```

### ⚠️ 주의사항
- 실제 Solar Pro 2 API를 호출합니다 
- API Key가 올바르게 설정되어 있어야 합니다
- **Assistant 모델의 응답은 `<think>...</think>` 태그가 반드시 포함되어야 합니다**
  - API 호출 시 `include_think=true` 파라미터를 사용하여 태그가 포함된 응답을 받습니다
  - **만약 Assistant 응답에 think 태그가 없으면 후처리 로직 추가가 필요합니다.**
    - 후처리: assistant 응답의 reasoning 부분을 추출하여 `<think>...</think>` 태그로 감싸야 합니다
    - 이는 모델 학습용 데이터 구축의 완성도를 위해 반드시 필요한 요소입니다

## 2-1. Single-turn & Multi-tool-use QA 시나리오 생성

### 📝 Single-turn이란?
- **한 번의 사용자 요청** → **한 번의 AI 답변**
- 중간에 여러 Tool을 호출할 수 있음
- 추가 질문 없이 한 번에 완결

### 🔧 Multi-tool-use란?
- 한 번의 요청을 해결하기 위해 **2개 이상의 Tool을 사용**
- Tool 호출이 순차적 또는 병렬로 실행됨

### 💡 예시
사용자: "한화시스템이 레이더/전자전 도메인을 다루는지 확인하고, 천궁(M-SAM)의 역할도 알려줘."

AI 사고 과정:
1. Tool 1 호출: defense_company_info(query="한화시스템")
2. Tool 2 호출: rok_weapon_info(keyword="천궁")
3. 결과 통합하여 최종 답변 생성
`


In [ ]:
# ============================================================
# Solar Pro 2 API를 활용한 합성 데이터셋 구축 : Single Turn   
# ============================================================

BASE_URL = "https://api.upstage.ai/v1"

# ============================================================
# Tool 실행 헬퍼 함수
# ============================================================

def _safe_json(obj) -> str:
    """JSON 직렬화 헬퍼"""
    try:
        return json.dumps(obj, ensure_ascii=False)
    except Exception:
        return json.dumps({"_error": "non-serializable"}, ensure_ascii=False)

def call_solar_api(messages: List[Dict[str, Any]], 
                   tools_spec: list,
                   temperature: float = 0.1, 
                   max_tokens: int = 800,
                   max_retries: int = 3) -> Dict[str, Any]:
    """
    Solar Pro 2 Chat Completions API 호출 (재시도 로직 포함)
    """
    url = f"{BASE_URL}/solar/chat/completions?include_think=true" # <think>...</think> 태그 추가 
    headers = {
        "Authorization": f"Bearer {UPSTAGE_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "solar-pro2",
        "messages": messages,
        "tools": tools_spec,
        "tool_choice": "auto",
        "temperature": temperature,
        "reasoning_effort" : "high", 
        "max_tokens": max_tokens,
    }
    
    for attempt in range(1, max_retries + 1):
        try:
            r = requests.post(url, headers=headers, json=payload, timeout=60)
            if r.status_code in (429,) or r.status_code >= 500:
                raise requests.HTTPError(f"{r.status_code} {r.text[:300]}")
            r.raise_for_status()
            return r.json()
        except requests.HTTPError as e:
            if attempt < max_retries:
                wait_time = (2 ** (attempt - 1)) + random.random()
                print(f"   ⚠️  재시도 {attempt}/{max_retries} (대기: {wait_time:.1f}초)")
                time.sleep(wait_time)
            else:
                print(f"   ❌ API 호출 실패: {e}")
                raise


def execute_tool(name: str, args: dict) -> str:
    """
    실제 Tool 실행 (도메인에 맞게 수정 필요)
    """
    if name == "defense_company_info":
        res = defense_company_info(**args)
    elif name == "rok_weapon_info":
        res = rok_weapon_info(**args)
    else:
        res = {"error": f"unknown tool: {name}"}
    
    out = _safe_json(res)
    # 길이 제한 (API 오류 방지)
    if len(out) > 4000:
        out = out[:4000] + f' ... (truncated {len(out)-4000} chars)'
    return out


print("✅ Solar Pro 2 API 설정 완료") 



# ============================================================
# Single-turn 시나리오 생성 함수
# ============================================================

def generate_single_turn_scenario(user_query: str,
                                   task_description: str,
                                   tools_spec: list,
                                   row_idx: Optional[int] = None) -> Dict[str, Any]:
    """
    Single-turn & Multi-tool-use 시나리오 생성.

    prompt 그리고 시나리오 생성 로직 등을 변경해보면서 자신만의 시나리오를 생성해보세요! 
    """
    system_prompt = (
        """
        너는 어떤 도구를 사용해야할 지 판단하고, 필요한 도구를 호출하는 도구 사용 전략가야.
        도구 인자는 스키마에 맞춰라. 여러 도구를 사용해야 할 경우 순차적으로 호출하라.
        항상 너의 사고 과정을 <think>...</think> 태그 안에 명시적으로 작성한 후, 최종 답변을 작성해야해.
        절대 내부 reasoning을 숨기지 말고 반드시 <think>...</think> 태그를 포함해 출력하세요.
        예시:
        <think>
        사용자의 요청을 단계별로 처리해보겠습니다. 
        "사용자 요청 분석: [사용자가 무엇을 원하는지 자세히 분석]\n"
        "필요한 도구 판단: [어떤 도구가 필요한지와 각 도구를 선택한 이유]\n"
        "각 도구 파라미터 설정: [각 도구의 파라미터를 어떻게 설정할지 상세히]\n"
        "도구 호출 순서 및 이유: [여러 도구를 호출한다면 순서와 각 순서의 이유]\n"
        </think>
        """
    )
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ]
    
    # messages 배열 (샘플 형식에 맞춤)
    messages_list = [
        {
            "role": "user",
            "content": user_query,
            "tool_calls": []
        }
    ]
    
    # 최대 5번 반복 (도구 호출)
    for step in range(5):
        try:
            data = call_solar_api(messages, tools_spec)
            msg = data["choices"][0]["message"]
            
            # Tool calls 추출
            tool_calls = []
            if isinstance(msg.get("tool_calls"), list):
                for tc in msg["tool_calls"]:
                    fn = tc.get("function") or {}
                    name = fn.get("name") or tc.get("name")
                    args_raw = fn.get("arguments") or tc.get("arguments") or "{}"
                    try:
                        args = json.loads(args_raw) if isinstance(args_raw, str) else args_raw
                    except Exception:
                        args = {}
                    tool_calls.append({"id": tc.get("id"), "name": name, "arguments": args})
            
            content = msg.get("content") or ""
            
            # Assistant 메시지 형식 구성
            assistant_tool_calls = []
            if tool_calls:
                assistant_tool_calls = [
                    {
                        "id": tc["id"],
                        "type": "function",
                        "function": {
                            "name": tc["name"],
                            "arguments": json.dumps(tc["arguments"], ensure_ascii=False)
                        }
                    } for tc in tool_calls
                ]
            
            # Assistant 메시지 추가
            messages_list.append({
                "role": "assistant",
                "content": content,
                "tool_calls": assistant_tool_calls
            })
            messages.append(msg)
            
            # 도구 호출이 없으면 최종 답변으로 종료
            if not tool_calls:
                break
            
            # 각 도구 실행 및 tool 메시지 추가
            for tc in tool_calls:
                result = execute_tool(tc["name"], tc.get("arguments", {}))
                
                tool_message = {
                    "role": "tool",
                    "tool_call_id": tc["id"],
                    "content": result
                }
                messages.append(tool_message)
                
                # 샘플 형식에 맞춰 tool 메시지 추가 (tool_calls 필드 포함)
                messages_list.append({
                    "role": "tool",
                    "content": result,
                    "tool_calls": []
                })
            
            time.sleep(0.5)
            
        except Exception as e:
            print(f"   ❌ 오류 발생: {e}")
            break
    
    # metadata를 JSON 문자열로 변환
    metadata_dict = {"tools": tools_spec}
    metadata_str = json.dumps(metadata_dict, ensure_ascii=False)
    
    # 샘플 형식에 맞춰 반환
    result = {
        "row": {
            "messages": messages_list,
            "metadata": metadata_str
        }
    }
    
    # row_idx가 제공된 경우에만 추가
    if row_idx is not None:
        result["row_idx"] = row_idx
    
    return result


def save_jsonl(items: List[Dict[str, Any]], path: str):
    os.makedirs(os.path.dirname(path) or ".", exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        for it in items:
            f.write(json.dumps(it, ensure_ascii=False) + "\n")
    print(f"✅ saved: {path}")

✅ Solar Pro 2 API 설정 완료


In [15]:
# ============================================================
# Single-turn 시나리오 생성 실행
# ============================================================
print("\n[1/2] Single-turn & Multi-tool-use 시나리오 생성 중...")
single_scenarios = []

# 시나리오 1
print("   [1-1] 시나리오 1 생성...")
scenario1 = generate_single_turn_scenario(
    user_query="한화시스템이 레이더/전자전 도메인을 다루는지 확인하고, 천궁(M-SAM)의 역할도 알려줘.",
    task_description="방산 업체 정보와 무기체계 정보를 통합 조회",
    tools_spec=tools,
    row_idx=1
)
single_scenarios.append(scenario1)
print(f"   ✅ 생성 완료 (row_idx: {scenario1.get('row_idx', 'N/A')})")

# 시나리오 2
print("   [1-2] 시나리오 2 생성...")
scenario2 = generate_single_turn_scenario(
    user_query="LIG Nex1의 감시정찰 여부를 확인하고, KF-21 보라매의 개발 현황도 알려줘.",
    task_description="방산 업체 도메인 확인 및 관련 무기체계 조회",
    tools_spec=tools,
    row_idx=2
)
single_scenarios.append(scenario2)
print(f"   ✅ 생성 완료 (row_idx: {scenario2.get('row_idx', 'N/A')})")

# 저장
save_jsonl(single_scenarios, "data/defense_single_turn_solar.jsonl")


[1/2] Single-turn & Multi-tool-use 시나리오 생성 중...
   [1-1] 시나리오 1 생성...
   ✅ 생성 완료 (row_idx: 1)
   [1-2] 시나리오 2 생성...
   ✅ 생성 완료 (row_idx: 2)
✅ saved: data/defense_single_turn_solar.jsonl


## 2-2. Multi-turn & Multi-tool-use 시나리오 생성

### 📝 Multi-turn이란?
- **여러 번의 대화 주고받기**
- 사용자 요청 → AI 답변 → 사용자 추가 질문 → AI 답변 (반복)
- 이전 맥락을 유지하면서 대화 진행

### 🔧 Multi-tool-use + Multi-turn
- 여러 턴에 걸쳐 다양한 Tool을 사용
- **오류 복구 시나리오** 포함 (의도적으로 실패 → 재시도)
- **조건부 선택** 가능

### 💡 예시
```
턴 1)
사용자: "한화시스템이 레이더/전자전 도메인을 하는지 먼저 확인해줘."
AI: defense_company_info() 호출 → 답변

턴 2)
사용자: "그럼 중거리 방공체계 천궁의 개요도 알려줘."
AI: rok_weapon_info() 호출 (실패) → 재시도 → 성공 → 답변

턴 3)
사용자: "천궁을 만든 회사 정보도 같이 보여줘."
AI: defense_company_info() 호출 → 답변
```

In [ ]:
# ============================================================
# Multi-turn 시나리오 생성 함수
# ============================================================

def generate_multi_turn_scenario(initial_query: str,
                                  follow_up_queries: List[str],
                                  task_description: str,
                                  tools_spec: list,
                                  row_idx: Optional[int] = None,
                                  max_steps: int = 4) -> Dict[str, Any]:
    """
    Multi-turn & Multi-tool-use 시나리오 생성.
    prompt 그리고 시나리오 생성 로직 등을 변경해보면서 자신만의 시나리오를 생성해보세요! 

    """
    system_prompt = (
        """
        너는 한국어로 대화한다. 여러 턴에 걸쳐 도구를 호출한다.
        인자가 틀리면 원인을 설명하고 인자를 수정해 재시도하라.
        너는 어떤 도구를 사용해야할 지 판단하고, 필요한 도구를 호출하는 도구 사용 전략가야.
        도구 인자는 스키마에 맞춰라. 여러 도구를 사용해야 할 경우 순차적으로 호출하라.
        항상 너의 사고 과정을 <think>...</think> 태그 안에 명시적으로 작성한 후, 최종 답변을 작성해야해.
        절대 내부 reasoning을 숨기지 말고 반드시 <think>...</think> 태그를 포함해 출력하세요.
        예시:
        <think>
        사용자의 요청을 단계별로 처리해보겠습니다. 
        "사용자 요청 분석: [사용자가 무엇을 원하는지 자세히 분석]\n"
        "필요한 도구 판단: [어떤 도구가 필요한지와 각 도구를 선택한 이유]\n"
        "각 도구 파라미터 설정: [각 도구의 파라미터를 어떻게 설정할지 상세히]\n"
        "도구 호출 순서 및 이유: [여러 도구를 호출한다면 순서와 각 순서의 이유]\n"
        </think>
        """ 
    )
    
    messages = [{"role": "system", "content": system_prompt}]
    # messages 배열 (샘플 형식에 맞춤)
    messages_list = []
    
    all_queries = [initial_query] + follow_up_queries
    
    for turn_idx, query in enumerate(all_queries):
        # User 메시지 추가
        user_message = {
            "role": "user",
            "content": query,
            "tool_calls": []
        }
        messages_list.append(user_message)
        messages.append({"role": "user", "content": query})
        
        # 각 턴당 최대 3번 시도 (오류 복구)
        for attempt in range(3):
            try:
                data = call_solar_api(messages, tools_spec)
                msg = data["choices"][0]["message"]
                
                # Tool calls 추출
                tool_calls = []
                if isinstance(msg.get("tool_calls"), list):
                    for tc in msg["tool_calls"]:
                        fn = tc.get("function") or {}
                        name = fn.get("name") or tc.get("name")
                        args_raw = fn.get("arguments") or tc.get("arguments") or "{}"
                        try:
                            args = json.loads(args_raw) if isinstance(args_raw, str) else args_raw
                        except Exception:
                            args = {}
                        tool_calls.append({"id": tc.get("id"), "name": name, "arguments": args})
                
                content = msg.get("content") or ""
                
                # Assistant 메시지 형식 구성
                assistant_tool_calls = []
                if tool_calls:
                    assistant_tool_calls = [
                        {
                            "id": tc["id"],
                            "type": "function",
                            "function": {
                                "name": tc["name"],
                                "arguments": json.dumps(tc["arguments"], ensure_ascii=False)
                            }
                        } for tc in tool_calls
                    ]
                
                # Assistant 메시지 추가
                messages_list.append({
                    "role": "assistant",
                    "content": content,
                    "tool_calls": assistant_tool_calls
                })
                messages.append(msg)
                
                # 도구 호출이 없으면 최종 답변으로 종료
                if not tool_calls:
                    break
                
                # 각 도구 실행 및 tool 메시지 추가
                has_error = False
                for tc in tool_calls:
                    # 오류 복구 시나리오: 두 번째 턴의 첫 시도에서 의도적 실패
                    if turn_idx == 1 and attempt == 0:
                        result = json.dumps({
                            "error": "Invalid tool call",
                            "details": {
                                "missing": [],
                                "invalid": {
                                    "parameter": "Expected integer, got string"
                                }
                            }
                        }, ensure_ascii=False)
                        has_error = True
                    else:
                        result = execute_tool(tc["name"], tc.get("arguments", {}))
                    
                    tool_message = {
                        "role": "tool",
                        "tool_call_id": tc["id"],
                        "content": result
                    }
                    messages.append(tool_message)
                    
                    # 샘플 형식에 맞춰 tool 메시지 추가 (tool_calls 필드 포함)
                    messages_list.append({
                        "role": "tool",
                        "content": result,
                        "tool_calls": []
                    })
                
                # 에러가 있으면 재시도 (다음 API 호출에서 assistant가 에러를 분석하고 재시도)
                if has_error:
                    continue
                else:
                    break
                    
            except Exception as e:
                print(f"   ❌ 턴 {turn_idx+1} 오류: {e}")
                break
            
            time.sleep(0.5)
    
    # metadata를 JSON 문자열로 변환
    metadata_dict = {"tools": tools_spec}
    metadata_str = json.dumps(metadata_dict, ensure_ascii=False)
    
    # 샘플 형식에 맞춰 반환
    result = {
        "row": {
            "messages": messages_list,
            "metadata": metadata_str
        }
    }
    
    # row_idx가 제공된 경우에만 추가
    if row_idx is not None:
        result["row_idx"] = row_idx
    
    return result


In [17]:

# ============================================================
# Multi-turn 시나리오 생성 실행
# ============================================================
print("\n[2/2] Multi-turn & Multi-tool-use 시나리오 생성 중...")
multi_scenarios = []

# 시나리오 3
print("   [2-1] 시나리오 1 생성...")
scenario3 = generate_multi_turn_scenario(
    initial_query="한화시스템이 레이더/전자전 도메인을 하는지 먼저 확인해줘.",
    follow_up_queries=[
        "그럼 중거리 방공체계 천궁의 개요도 알려줘.",
        "천궁을 만든 회사 정보도 같이 보여줘."
    ],
    task_description="방산 업체 확인 후 관련 무기체계 정보를 단계적으로 조회",
    tools_spec=tools,
    row_idx=3
)
multi_scenarios.append(scenario3)
num_turns = len([m for m in scenario3['row']['messages'] if m.get('role') == 'user'])
print(f"   ✅ 생성 완료 (row_idx: {scenario3.get('row_idx', 'N/A')}, 턴: {num_turns})")

# 시나리오 4
print("   [2-2] 시나리오 2 생성...")
scenario4 = generate_multi_turn_scenario(
    initial_query="KF-21 보라매에 대해 알려줘.",
    follow_up_queries=[
        "이걸 만드는 회사의 채용 정보도 알고 싶어.",
        "그 회사가 다루는 다른 도메인도 알려줘."
    ],
    task_description="무기체계 조회 후 제조사 정보 및 상세 도메인 확인",
    tools_spec=tools,
    row_idx=4
)
multi_scenarios.append(scenario4)
num_turns = len([m for m in scenario4['row']['messages'] if m.get('role') == 'user'])
print(f"   ✅ 생성 완료 (row_idx: {scenario4.get('row_idx', 'N/A')}, 턴: {num_turns})")

# 저장
save_jsonl(multi_scenarios, "data/defense_multi_turn_solar.jsonl")




[2/2] Multi-turn & Multi-tool-use 시나리오 생성 중...
   [2-1] 시나리오 1 생성...
   ✅ 생성 완료 (row_idx: 3, 턴: 3)
   [2-2] 시나리오 2 생성...
   ✅ 생성 완료 (row_idx: 4, 턴: 3)
✅ saved: data/defense_multi_turn_solar.jsonl


# 3️⃣ BFCL 평가 (Berkeley Function Call Leaderboard)

이 섹션에서는 생성된 시나리오를 **BFCL 기준**으로 평가합니다.

### 📊 평가 기준 

- Single Turn

| 평가 항목 | 설명 | 방법 |
|----------|------|------|
| **Correct Function Name** | 올바른 함수 호출 여부 | Rule-based |
| **Valid Arguments** | 인자 타입/포맷 정확성 | Rule-based |
| **No Hallucinated Calls** | 정의되지 않은 함수 방지 | Rule-based |


### ✅ 통과 조건
- 각 평가 기준을 **모두** 통과해야 최종 데이터셋에 포함됩니다

### 📁 결과물
- **CSV**: 전체 평가 리포트 (`artifacts/bfcl_defense_report_multi.csv`)
- **JSONL**: 통과한 시나리오만 (`artifacts/bfcl_defense_passes_multi.jsonl`)

### 3-1 Single Turn 평가

evaluate.py를 사용하여 Single Turn 시나리오를 평가합니다.

In [18]:
# ============================================================
# evaluate.py 모듈 import
# ============================================================
import sys
from pathlib import Path

# eval 디렉토리를 path에 추가
eval_dir = Path("eval")
if str(eval_dir.absolute()) not in sys.path:
    sys.path.insert(0, str(eval_dir.absolute()))

# evaluate 모듈 import
from evaluate import evaluate_entry, load_data

print("✅ evaluate.py 모듈 로드 완료")


✅ evaluate.py 모듈 로드 완료


In [19]:
# ============================================================
# Single Turn 시나리오 평가
# ============================================================
print("\n" + "=" * 60)
print("📊 Single Turn 시나리오 평가")
print("=" * 60)

# Single turn 데이터 로드
single_turn_data = load_data("data/defense_single_turn_solar.jsonl")

single_turn_results = []
for entry in single_turn_data:
    entry_id = entry.get("row_idx") or entry.get("id", "unknown")
    print(f"\n[평가 중] ID: {entry_id}")
    
    try:
        result = evaluate_entry(entry)
        result["id"] = entry_id
        single_turn_results.append({
            "id": entry_id,
            **result
        })
        
        # 결과 출력
        cfn = '✅' if result['correct_function_name'] else '❌'
        va = '✅' if result['valid_arguments'] else '❌'
        nhc = '✅' if result['no_hallucinated_calls'] else '❌'
        pass_status = '✅ PASS' if result['pass'] else '❌ FAIL'
        
        print(f"   - Correct Function Name: {cfn}")
        print(f"   - Valid Arguments: {va}")
        print(f"   - No Hallucinated Calls: {nhc}")
        print(f"   - 최종 결과: {pass_status}")
        print(f"   - 턴 수: {result.get('num_turns', 0)}, 총 도구 호출: {result.get('total_tool_calls', 0)}")
        
        if result.get('errors'):
            print(f"   - 오류: {', '.join(result['errors'][:2])}")  # 처음 2개만 표시
        
    except Exception as e:
        print(f"   ❌ 평가 중 오류 발생: {e}")
        single_turn_results.append({
            "id": entry_id,
            "correct_function_name": False,
            "valid_arguments": False,
            "no_hallucinated_calls": False,
            "pass": False,
            "errors": [f"평가 중 오류 발생: {str(e)}"]
        })

# 요약 출력
total_single = len(single_turn_results)
passed_single = sum(1 for r in single_turn_results if r.get("pass"))

print("\n" + "=" * 60)
print("📊 Single Turn 평가 결과 요약")
print("=" * 60)
print(f"총 평가 항목: {total_single}")
print(f"✅ 통과: {passed_single} ({passed_single/total_single*100:.1f}%)" if total_single > 0 else "✅ 통과: 0")
print(f"❌ 실패: {total_single - passed_single} ({(total_single-passed_single)/total_single*100:.1f}%)" if total_single > 0 else "❌ 실패: 0")

if total_single > 0:
    print(f"\n📋 평가 항목별 통과율:")
    print(f"  - Correct Function Name: {sum(1 for r in single_turn_results if r.get('correct_function_name'))}/{total_single}")
    print(f"  - Valid Arguments: {sum(1 for r in single_turn_results if r.get('valid_arguments'))}/{total_single}")
    print(f"  - No Hallucinated Calls: {sum(1 for r in single_turn_results if r.get('no_hallucinated_calls'))}/{total_single}")
print("=" * 60)



📊 Single Turn 시나리오 평가

[평가 중] ID: 1
   - Correct Function Name: ✅
   - Valid Arguments: ✅
   - No Hallucinated Calls: ✅
   - 최종 결과: ✅ PASS
   - 턴 수: 1, 총 도구 호출: 2

[평가 중] ID: 2
   - Correct Function Name: ✅
   - Valid Arguments: ✅
   - No Hallucinated Calls: ✅
   - 최종 결과: ✅ PASS
   - 턴 수: 1, 총 도구 호출: 2

📊 Single Turn 평가 결과 요약
총 평가 항목: 2
✅ 통과: 2 (100.0%)
❌ 실패: 0 (0.0%)

📋 평가 항목별 통과율:
  - Correct Function Name: 2/2
  - Valid Arguments: 2/2
  - No Hallucinated Calls: 2/2


### 3-2 Multi Turn 평가

evaluate.py를 사용하여 Multi Turn 시나리오를 평가합니다.

In [20]:
# ============================================================
# Multi Turn 시나리오 평가
# ============================================================
print("\n" + "=" * 60)
print("📊 Multi Turn 시나리오 평가")
print("=" * 60)

# Multi turn 데이터 로드
multi_turn_data = load_data("data/defense_multi_turn_solar.jsonl")

multi_turn_results = []
for entry in multi_turn_data:
    entry_id = entry.get("row_idx") or entry.get("id", "unknown")
    print(f"\n[평가 중] ID: {entry_id}")
    
    try:
        result = evaluate_entry(entry)
        result["id"] = entry_id
        multi_turn_results.append({
            "id": entry_id,
            **result
        })
        
        # 결과 출력
        cfn = '✅' if result['correct_function_name'] else '❌'
        va = '✅' if result['valid_arguments'] else '❌'
        nhc = '✅' if result['no_hallucinated_calls'] else '❌'
        pass_status = '✅ PASS' if result['pass'] else '❌ FAIL'
        
        print(f"   - Correct Function Name: {cfn}")
        print(f"   - Valid Arguments: {va}")
        print(f"   - No Hallucinated Calls: {nhc}")
        print(f"   - 최종 결과: {pass_status}")
        print(f"   - 턴 수: {result.get('num_turns', 0)}, 총 도구 호출: {result.get('total_tool_calls', 0)}")
        
        # 턴별 결과 출력
        if result.get('turn_results'):
            print(f"   - 턴별 결과:")
            for turn_result in result['turn_results']:
                turn_num = turn_result.get('turn', 0)
                turn_pass = '✅' if turn_result.get('pass') else '❌'
                print(f"     * 턴 {turn_num}: {turn_pass} (도구 호출: {turn_result.get('num_tool_calls', 0)})")
        
        if result.get('errors'):
            print(f"   - 오류: {', '.join(result['errors'][:2])}")  # 처음 2개만 표시
        
    except Exception as e:
        print(f"   ❌ 평가 중 오류 발생: {e}")
        multi_turn_results.append({
            "id": entry_id,
            "correct_function_name": False,
            "valid_arguments": False,
            "no_hallucinated_calls": False,
            "pass": False,
            "errors": [f"평가 중 오류 발생: {str(e)}"]
        })

# 요약 출력
total_multi = len(multi_turn_results)
passed_multi = sum(1 for r in multi_turn_results if r.get("pass"))

print("\n" + "=" * 60)
print("📊 Multi Turn 평가 결과 요약")
print("=" * 60)
print(f"총 평가 항목: {total_multi}")
print(f"✅ 통과: {passed_multi} ({passed_multi/total_multi*100:.1f}%)" if total_multi > 0 else "✅ 통과: 0")
print(f"❌ 실패: {total_multi - passed_multi} ({(total_multi-passed_multi)/total_multi*100:.1f}%)" if total_multi > 0 else "❌ 실패: 0")

if total_multi > 0:
    print(f"\n📋 평가 항목별 통과율:")
    print(f"  - Correct Function Name: {sum(1 for r in multi_turn_results if r.get('correct_function_name'))}/{total_multi}")
    print(f"  - Valid Arguments: {sum(1 for r in multi_turn_results if r.get('valid_arguments'))}/{total_multi}")
    print(f"  - No Hallucinated Calls: {sum(1 for r in multi_turn_results if r.get('no_hallucinated_calls'))}/{total_multi}")
print("=" * 60)



📊 Multi Turn 시나리오 평가

[평가 중] ID: 3
   - Correct Function Name: ✅
   - Valid Arguments: ✅
   - No Hallucinated Calls: ✅
   - 최종 결과: ✅ PASS
   - 턴 수: 3, 총 도구 호출: 3
   - 턴별 결과:
     * 턴 1: ✅ (도구 호출: 1)
     * 턴 2: ✅ (도구 호출: 1)
     * 턴 3: ✅ (도구 호출: 1)

[평가 중] ID: 4
   - Correct Function Name: ✅
   - Valid Arguments: ✅
   - No Hallucinated Calls: ✅
   - 최종 결과: ✅ PASS
   - 턴 수: 4, 총 도구 호출: 4
   - 턴별 결과:
     * 턴 1: ✅ (도구 호출: 1)
     * 턴 2: ✅ (도구 호출: 1)
     * 턴 3: ✅ (도구 호출: 1)
     * 턴 4: ✅ (도구 호출: 1)

📊 Multi Turn 평가 결과 요약
총 평가 항목: 2
✅ 통과: 2 (100.0%)
❌ 실패: 0 (0.0%)

📋 평가 항목별 통과율:
  - Correct Function Name: 2/2
  - Valid Arguments: 2/2
  - No Hallucinated Calls: 2/2


# 4️⃣ 최종 데이터셋 평가 & 저장

이 섹션에서는 생성된 **Single-turn과 Multi-turn 시나리오를 모두 평가**하고 데이터셋을 저장합니다.
해당 데이터셋을 바탕으로 정성평가를 진행합니다

### 📊 정성 평가 기준
- 정량 평가 기준 통과한 데이터셋을 바탕으로 정성 평가를 진행합니다.

| 평가 항목         | 배점 | 평가 기준                                                                                                       |
| ------------- | -- | ----------------------------------------------------------------------------------------------------------- |
| **시나리오 적합성**  | 15 | - 도메인/Task와 일관성 있는가?<br>- 실제 서비스/공공 API 활용 맥락을 반영했는가?                                                       |
| **창의성 및 다양성** | 15 | - 단순 반복적인 QA가 아닌 새로운 활용 아이디어를 제시했는가?<br>- 시나리오가 다양한 상황·조건(오류, 조건 분기, 동적 검색 등)을 폭넓게 커버했는가?                   |
| **구체성**       | 15 | - 상황·조건(시간/위치/제약 등)이 구체적으로 기술되었는가?<br>- 실제 API 호출로 매핑 가능성이 명확한가?                                            |
| **실행 가능성**    | 25 | - LLM + 한국 API 기반으로 실제 실행 가능한 수준인가?<br>- 너무 추상적이거나 불가능한 요청은 아닌가?<br>- 실패(에러, 잘못된 호출) 이후 성공적으로 복구했는가? *(중요)* |
| **윤리·안전성**    | 10 | - 개인정보, 정치/사회적 민감 이슈를 피했는가?<br>- AI 윤리적 문제(차별, 편향 등) 우려가 없는가?                                               |
                                |






### 📁 최종 결과물
- `outputs/final_dataset.jsonl`: 정량평가 통과한 시나리오 
- `outputs/final_dataset.json`: 데이터셋 + 통계
- `artifacts/bfcl_evaluation_report.csv`: 정량 평가 리포트


In [22]:
# ============================================================
# 최종 데이터셋 평가 & 저장
# ============================================================

print("\n" + "=" * 60)
print("📊 전체 평가 결과 통합")
print("=" * 60)

# 모든 평가 결과 통합
all_results = single_turn_results + multi_turn_results
all_evaluation_results = []

# 통과한 시나리오만 원본 데이터와 매칭
passed_scenarios = []

# Single turn 통과 항목
for result in single_turn_results:
    if result.get("pass"):
        # 원본 데이터 찾기
        entry_id = result.get("id")
        original_entry = next((e for e in single_turn_data if (e.get("row_idx") or e.get("id")) == entry_id), None)
        if original_entry:
            original_entry["evaluation"] = result
            passed_scenarios.append(original_entry)

# Multi turn 통과 항목
for result in multi_turn_results:
    if result.get("pass"):
        # 원본 데이터 찾기
        entry_id = result.get("id")
        original_entry = next((e for e in multi_turn_data if (e.get("row_idx") or e.get("id")) == entry_id), None)
        if original_entry:
            original_entry["evaluation"] = result
            passed_scenarios.append(original_entry)

# 평가 결과 포맷팅 (CSV용)
for result in all_results:
    all_evaluation_results.append({
        "id": result.get("id"),
        "correct_function_name": result.get("correct_function_name", False),
        "valid_arguments": result.get("valid_arguments", False),
        "no_hallucinated_calls": result.get("no_hallucinated_calls", False),
        "pass": result.get("pass", False),
        "num_turns": result.get("num_turns", 0),
        "total_tool_calls": result.get("total_tool_calls", 0)
    })

# 통계 계산
total = len(all_results)
passed = sum(1 for r in all_results if r.get("pass"))
failed = total - passed

single_turn_passed = sum(1 for r in single_turn_results if r.get("pass"))
multi_turn_passed = sum(1 for r in multi_turn_results if r.get("pass"))

print(f"전체 시나리오: {total}개")
print(f"  - Single Turn: {len(single_turn_results)}개 (통과: {single_turn_passed}개)")
print(f"  - Multi Turn: {len(multi_turn_results)}개 (통과: {multi_turn_passed}개)")
print(f"\n✅ 총 통과: {passed}개 ({passed/total*100:.1f}%)" if total > 0 else "\n✅ 총 통과: 0개")
print(f"❌ 총 실패: {failed}개 ({failed/total*100:.1f}%)" if total > 0 else "❌ 총 실패: 0개")
print("=" * 60)

# --------------------------
# 최종 데이터셋 저장
# --------------------------
print("\n" + "=" * 60)
print("💾 최종 데이터셋 저장")
print("=" * 60)

# 1. JSONL 형식으로 통과 항목만 저장
output_jsonl = "outputs/final_dataset.jsonl"
with open(output_jsonl, "w", encoding="utf-8") as f:
    for scenario in passed_scenarios:
        f.write(json.dumps(scenario, ensure_ascii=False) + "\n")
print(f"✅ JSONL 저장: {output_jsonl} (통과 항목: {len(passed_scenarios)}개)")

# 2. JSON 형식으로 전체 저장
final_dataset = {
    "project": "Upstage AI Agent QA 시나리오 데이터셋 구축",
    "domain": SELECTED_DOMAIN,
    "created_at": datetime.now().isoformat(),
    "tools": tools,
    "scenarios": passed_scenarios,
    "statistics": {
        "total_generated": total,
        "passed_evaluation": len(passed_scenarios),
        "single_turn_total": len(single_turn_results),
        "single_turn_passed": single_turn_passed,
        "multi_turn_total": len(multi_turn_results),
        "multi_turn_passed": multi_turn_passed,
        "pass_rate": f"{passed/total*100:.1f}%" if total > 0 else "0%"
    }
}

output_json = "outputs/final_dataset.json"
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(final_dataset, f, ensure_ascii=False, indent=2)
print(f"✅ JSON 저장: {output_json}")

# 3. 평가 리포트 CSV 저장
report_csv = "artifacts/bfcl_evaluation_report.csv"
with open(report_csv, "w", newline="", encoding="utf-8") as f:
    fieldnames = [
        "id", "correct_function_name", "valid_arguments", 
        "no_hallucinated_calls", "pass", "num_turns", "total_tool_calls"
    ]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for result in all_evaluation_results:
        row = {
            "id": result.get("id"),
            "correct_function_name": 1 if result.get("correct_function_name") else 0,
            "valid_arguments": 1 if result.get("valid_arguments") else 0,
            "no_hallucinated_calls": 1 if result.get("no_hallucinated_calls") else 0,
            "pass": 1 if result.get("pass") else 0,
            "num_turns": result.get("num_turns", 0),
            "total_tool_calls": result.get("total_tool_calls", 0)
        }
        writer.writerow(row)
print(f"✅ CSV 리포트: {report_csv}")

print("\n" + "=" * 60)
print("🎉 모든 작업 완료!")
print("=" * 60)
print(f"\n📁 생성된 파일:")
print(f"   1. {output_jsonl} ({len(passed_scenarios)}개 통과 항목)")
print(f"   2. {output_json}")
print(f"   3. {report_csv}")
print(f"\n📊 최종 통계:")
print(f"   - Single-turn 통과: {single_turn_passed}/{len(single_turn_results)}개")
print(f"   - Multi-turn 통과: {multi_turn_passed}/{len(multi_turn_results)}개")
print(f"   - 전체 통과율: {passed/total*100:.1f}%" if total > 0 else "   - 전체 통과율: 0%")
print("\n✨정성평가를 진행보세요!")



📊 전체 평가 결과 통합
전체 시나리오: 4개
  - Single Turn: 2개 (통과: 2개)
  - Multi Turn: 2개 (통과: 2개)

✅ 총 통과: 4개 (100.0%)
❌ 총 실패: 0개 (0.0%)

💾 최종 데이터셋 저장
✅ JSONL 저장: outputs/final_dataset.jsonl (통과 항목: 4개)
✅ JSON 저장: outputs/final_dataset.json
✅ CSV 리포트: artifacts/bfcl_evaluation_report.csv

🎉 모든 작업 완료!

📁 생성된 파일:
   1. outputs/final_dataset.jsonl (4개 통과 항목)
   2. outputs/final_dataset.json
   3. artifacts/bfcl_evaluation_report.csv

📊 최종 통계:
   - Single-turn 통과: 2/2개
   - Multi-turn 통과: 2/2개
   - 전체 통과율: 100.0%

✨정성평가를 진행보세요!
